In [ ]:
# Importing Libraries
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import datetime

import statsmodels.tsa.api as smt
import statsmodels.api as sm
from statsmodels.tools.eval_measures import rmse
import pickle

### Regressive Modeling

In [ ]:
def generate_supervised(data):
    supervised = data.copy()
    
    # Creating column for each lag
    for i in range(1,13):
        col_name = 'lag_' + str(i)
        supervised[col_name] = supervised['sales_diff'].shift(i)
    
    # Dropping null values
    supervised = supervised.dropna().reset_index(drop=True)
    
    supervised.to_csv('../data/input/model.csv', index=False)
    
    return supervised

In [ ]:
stationary = pd.read_csv('../data/input/stationary.csv')
model_df = generate_supervised(stationary)

### ARIMA Modeling

In [ ]:
def generate_arima_data(data):
    dt_data = data.set_index('orderDate').drop('sales', axis=1)
    dt_data.dropna(axis=0)
    
    dt_data.to_csv('../data/output/arima.csv')
    
    return dt_data

In [ ]:
datetime_df = generate_arima_data(stationary)
datetime_df

#### Loading the ARIMA Data

In [ ]:
def load_data():
    return pd.read_csv('../data/output/arima.csv').set_index('orderDate')

arm_data = load_data()

In [ ]:
arm_data.index = pd.to_datetime(arm_data.index)

### SARIMAX Modeling

In [ ]:
def get_scores(data):
    
    model_scores = {}
    
    rmse = np.sqrt(mean_squared_error(data.sales_diff[-12:], data.forecast[-12:]))
    mae = mean_absolute_error(data.sales_diff[-12:], data.forecast[-12:])
    r2 = r2_score(data.sales_diff[-12:], data.forecast[-12:])
    model_scores['ARIMA'] = [rmse, mae, r2]
    
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R2 Score: {r2}")
    
    pickle.dump(model_scores, open( "arima_model_scores.p", "wb" ))

In [ ]:
def sarimax_model(data):
    # Model
    sar = sm.tsa.statespace.SARIMAX(arm_data.sales_diff, order=(12,0,0), seasonal_order=(0,1,0,12), trend='c').fit()

    # Predictions
    start, end, dynamic = 40, 100, 7
    data['forecast'] = sar.predict(start=start, end=end, dynamic=dynamic) 
    pred_df = data.forecast[start+dynamic:end]
    
    data[['sales_diff', 'forecast']].plot(color=['mediumblue', 'Red'])
    
    get_scores(data)

    return sar, data, pred_df

sar, arm_data, predictions = sarimax_model(arm_data)

In [ ]:
sar.plot_diagnostics(figsize=(10, 8));

### Conclusion

In [ ]:
def predict_df(prediction_df):
    
    #load in original dataframe without scaling applied
    original_df = pd.read_csv('../data/input/train.csv')
    original_df.date = original_df.date.apply(lambda x: str(x)[:-3])
    original_df = original_df.groupby('orderDate')['sales'].sum().reset_index()
    original_df.date = pd.to_datetime(original_df.date)
    
    #create dataframe that shows the predicted sales
    result_list = []
    sales_dates = list(original_df[-13:].date)
    act_sales = list(original_df[-13:].sales)
    
    for index in range(0,len(prediction_df)):
        result_dict = {}
        result_dict['pred_value'] = int(prediction_df[index] + act_sales[index])
        result_dict['date'] = sales_dates[index+1]
        result_list.append(result_dict)
        
    df_result = pd.DataFrame(result_list)
    
    return df_result, original_df

In [ ]:
def plot_results(results, original_df, model_name):

    fig, ax = plt.subplots(figsize=(15,5))
    sns.lineplot(original_df.date, original_df.sales, data=original_df, ax=ax, 
                label='Original', color='mediumblue')
    sns.lineplot(results.date, results.pred_value, data=results, ax=ax, 
                 label='Predicted', color='Red')
    
    ax.set(xlabel = "Date",
           ylabel = "Orders",
           title = f"{model_name} Order Forecaster")
    
    ax.legend()
    
    sns.despine()
    

    plt.savefig(f'../model_output/{model_name}_forecast.png')

In [ ]:
prediction_df, original_df = predict_df(predictions)
plot_results(prediction_df, original_df, 'arima')